# 載入函式庫與資料集

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
np.random.seed(627)  # 指定亂數種子：只要不更改，每次運作時即可保持再現性
# 載入資料集
df = pd.read_csv("./titanic_data.csv")

In [ ]:
# 查看前5筆記錄
df.head()

鐵達尼號乘客資料集 各欄位說明可參考：
*   https://www.kaggle.com/competitions/titanic/data
*   https://ithelp.ithome.com.tw/articles/10200327


In [ ]:
# 顯示資料集的形狀
df.shape

In [ ]:
# 顯示資料集的描述資料
df.describe()

In [ ]:
# 顯示資料集的資訊
df.info()

In [ ]:
# 顯示沒有資料的筆數
df.isnull().sum()

# 資料前處理
> 對於真實世界的資料，前處理十分關鍵，也非常需要花精神

In [ ]:
# 刪除不需要的欄位
df = df.drop(["name", "ticket", "cabin"], axis=1)

In [ ]:
# 處理遺失資料
df[["age"]] = df[["age"]].fillna(value=df[["age"]].mean())
df[["fare"]] = df[["fare"]].fillna(value=df[["fare"]].mean())

df[["embarked"]] = df[["embarked"]].fillna(value=df["embarked"].value_counts().idxmax())
print(df["embarked"].value_counts())
print(df["embarked"].value_counts().idxmax())

In [ ]:
# 轉換分類資料
df["sex"] = df["sex"].map( {"female": 1, "male": 0} ).astype(int)

In [ ]:
# Embarked欄位的One-hot編碼
enbarked_one_hot = pd.get_dummies(df["embarked"], prefix="embarked")
df = df.drop("embarked", axis=1)
df = df.join(enbarked_one_hot)

In [ ]:
# 將標籤的 survived 欄位移至最後
df_survived = df.pop("survived") 
df["survived"] = df_survived

In [ ]:
df.head()

In [ ]:
# 分割成訓練(80%)和測試(20%)資料集
mask = np.random.rand(len(df)) < 0.8
df_train = df[mask]
df_test = df[~mask]
print("Train:", df_train.shape)
print("Test:", df_test.shape)
# 儲存處理後的資料
df_train.to_csv("titanic_train.csv", index=False)
df_test.to_csv("titanic_test.csv", index=False)

# 載入已經預處理好的資料
> 上述資料集整理處理完，往後即能直接載入資料集，不需每次都要重新處理原始資料

In [ ]:
np.random.seed(627)  # 指定亂數種子：只要不更改，每次運作時即可保持再現性
# 載入Titanic的訓練和測試資料集
df_train = pd.read_csv("./titanic_train.csv")
df_test = pd.read_csv("./titanic_test.csv")
dataset_train = df_train.values
dataset_test = df_test.values

In [ ]:
# 分割成特徵資料和標籤資料
X_train = dataset_train[:, 0:9]
Y_train = dataset_train[:, 9]
X_test = dataset_test[:, 0:9]
Y_test = dataset_test[:, 9]
# 特徵標準化
X_train -= X_train.mean(axis=0)
X_train /= X_train.std(axis=0)
X_test -= X_test.mean(axis=0)
X_test /= X_test.std(axis=0)

# 載入先前訓練過的模型（H5格式）

> 若你有先前訓練並存檔的模型，在此載入。若無，則不需執行此cell



In [ ]:
# 載入模型
from tensorflow.keras.models import load_model

model = Sequential()
model = load_model("你的學號.h5")

# 建立模型
> 若已載入先前訓練過的模型，則此處不要執行。

In [ ]:
# 定義模型
model = Sequential()
model.add(Dense(11, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(11, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()   # 顯示模型摘要資訊

# 訓練神經網路

In [ ]:
# 編譯模型
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# 訓練模型
print("Training ...")
history = model.fit(X_train, Y_train, validation_split=0.2, epochs=100, batch_size=32)

In [ ]:
# 評估模型
print("\nTesting ...")
loss, accuracy = model.evaluate(X_train, Y_train, verbose=0)
print("訓練資料集的準確度 = {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print("測試資料集的準確度 = {:.2f}".format(accuracy))

In [ ]:
# 顯示圖表來分析模型的訓練過程
import matplotlib.pyplot as plt
# 顯示訓練和驗證損失
loss = history.history["loss"]
epochs = range(1, len(loss)+1)
val_loss = history.history["val_loss"]
plt.plot(epochs, loss, "b-", label="Training Loss")
plt.plot(epochs, val_loss, "r--", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()
# 顯示訓練和驗證準確度
acc = history.history["accuracy"]
epochs = range(1, len(acc)+1)
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "b-", label="Training Acc")
plt.plot(epochs, val_acc, "r--", label="Validation Acc")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# 重新執行模型定義與訓練
> 從上述歷程追蹤來觀察適合的迭代次數，避免過度學習造成overfitting

In [ ]:
model = Sequential()
model.add(Dense(11, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(11, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, Y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=0)  # epochs改為適當的迭代次數

In [ ]:
# 評估模型
print("\nTesting ...")
loss, accuracy = model.evaluate(X_train, Y_train, verbose=0)
print("訓練資料集的準確度 = {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print("測試資料集的準確度 = {:.2f}".format(accuracy))

In [ ]:
# 儲存Keras模型
print("Saving Model ...")
model.save("你的學號.h5")

In [ ]:
# 計算分類的預測值
print("\nPredicting ...")
Y_pred = (model.predict(X_test) > 0.5).astype("int32") # 預測機率轉換成0或1，大於0.5為1、小於等於0.5為0
print(Y_pred.reshape(-1))
print(Y_test.astype(int))

In [ ]:
# 顯示混淆矩陣
tb = pd.crosstab(Y_test.astype(int), Y_pred.reshape(-1), rownames=["label"], colnames=["predict"])
tb

In [ ]:
# 儲存Keras模型
print("Saving Model ...")
model.save("你的學號.h5")